<a href="https://colab.research.google.com/github/StellarLuminosity/Sustainability-for-Business-Efficiency/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GenAI Capstone – Sustainability Operations for Business Efficiency Advisor
**Predicting the impact of operational inefficiencies and generating actionable, SDG-aligned solutions**

## Overview
This notebook builds a generative AI agent workflow that:
- Identifies and classifies a business inefficiencies (Inventory Waste or Energy Inefficiency)
- Stores and retrieves relevant cases using embeddings and ChromaDB
- Estimates cost of inaction for the business inefficiency
- Suggests SDG-aligned, actionable solutions that will save business costs and align with sustainability practices
- Generates a business-ready report


## -- Setup --
Install required packages and import libraries.

In [ ]:
!pip install langchain-community langchain pdfplumber langgraph langchainhub chromadb openai langchain-google-genai google-generativeai tqdm kagglehub[pandas-datasets] sentence-transformers --quiet

In [ ]:
# Imports

import os
import re
import json
import uuid
import shutil
import requests
import pdfplumber
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chromadb
import kagglehub
from google import genai
from google.genai import types
from bs4 import BeautifulSoup
from chromadb.config import Settings
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.genai.types import EmbedContentConfig
from IPython.display import HTML, Markdown, display
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from typing import Optional, List, Literal, TypedDict, Dict, Any
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from kagglehub import KaggleDatasetAdapter
from langchain.schema import Document
from langgraph.prebuilt import ToolNode
from pydantic import BaseModel, Field
from google.api_core import retry
from tqdm import tqdm

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from typing import TypedDict, List, Optional, Literal
from langchain_core.documents import Document

## -- API and Gemini Setup --

In [ ]:
# wraps the Gemini model’s generate_content method with automatic retry logic if it hits temporary errors — like rate limits or backend hiccups
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
client = genai.Client(api_key=GOOGLE_API_KEY)

## -- Chroma DB Setup --

In [ ]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __init__(self, client, document_mode=True):
        self.client = client
        self.document_mode = document_mode

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: List[str]) -> List[List[float]]:
        task_type = "retrieval_document" if self.document_mode else "retrieval_query"
        response = self.client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=EmbedContentConfig(task_type=task_type),
        )
        return [e.values for e in response.embeddings]

In [ ]:
# initialize ChromaDB In-Memory Client
chroma_client = chromadb.Client()

In [ ]:
# Instantiate the embedding function
embed_fn = GeminiEmbeddingFunction(client=client, document_mode=True)

# Create collections (in-memory, no disk persistence)
inventory_collection = chroma_client.get_or_create_collection(
    name="inventory_cases",
    embedding_function=embed_fn,
)

energy_collection = chroma_client.get_or_create_collection(
    name="energy_cases",
    embedding_function=embed_fn,
)

## -- Adding Data to ChromaDB --

In [ ]:
# Metadata for each uploaded document
# {
#   "purpose": ["case_study", "cost_estimates", "solution", "mixed"],
#   "domain": ["inventory", "energy"]
# }


### --- Inventory Case Studies ---

In [ ]:
url = "https://www.ccmonet.ai/blog-posts-new/inventory-management-case-studies-real-success-stories"
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "html.parser")
content_tags = soup.find_all(['p', 'li', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

full_text = '\n'.join([tag.get_text(strip=True) for tag in content_tags])
full_text = full_text[2733:] # start only from the relevant index

In [ ]:
# List of headers
section_headers = [
    "Case Study 1: Amazon's Revolutionary Inventory System",
    "Case Study 2: Walmart's Supply Chain Mastery",
    "Case Study 3: Zara's Fast Fashion Inventory Approach",
    "Case Study 4: Coca-Cola's Global Distribution Network",
    "Emerging Trends in Inventory Management",
    "Key Takeaways from Inventory Management Case Studies",
    "Future of Inventory Management: Predictions and Insights",
    "Conclusion: Learning from Success Stories",
    "FAQ"
]

# Create a header regex pattern
escaped_headers = [re.escape(header) for header in section_headers]
pattern = f"({'|'.join(escaped_headers)})"

# Split by header
parts = re.split(pattern, full_text)

# Combine each header with its following content
chunks = []
for i in range(1, len(parts), 2):  # step by 2 to pair header with body
    header = parts[i].strip()
    body = parts[i + 1].strip() if i + 1 < len(parts) else ""
    chunk = f"{header}\n{body}"
    chunks.append(chunk)

# Print the chunks
# for i, chunk in enumerate(chunks):
#     print(f"\n\nChunk {i}:\n{chunk}")

In [ ]:
# Convert chunks in LangChain documents

docs = [
    Document(
        page_content=chunk,
        metadata={
            "purpose": "case_study",
            "domain": "inventory"
        },
    )
    for chunk in chunks
]


Insert into ChromaDB

In [ ]:
# extract and format for ChromaDB
documents = [doc.page_content for doc in docs]
metadatas = [doc.metadata for doc in docs]
ids = [str(uuid.uuid4()) for _ in docs]

In [ ]:
inventory_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)
# inventory_collection.peek()

### --- Inventory Optimization: Case Study ---

In [ ]:
url = 'https://statso.io/wp-content/uploads/2023/08/demand_inventory.csv'
inv_opt = pd.read_csv(url)

In [ ]:
# Data pre-processing
# Removing the unnecessary 'Unnamed: 0' column
inv_opt.drop(columns=['Unnamed: 0'], inplace=True)

# Converting 'Data' to suitable format and using it as the index
inv_opt['Date'] = pd.to_datetime(inv_opt['Date'])
inv_opt.set_index('Date', inplace=True)

# Adding rolling average demand, computed over the last 7 days
inv_opt['rolling_demand'] = inv_opt['Demand'].rolling(window=7, min_periods=1).mean()

# Adding days of inventory
inv_opt['days_of_inventory'] = inv_opt['Inventory'] / inv_opt['rolling_demand']

In [ ]:
# Visualizing the Data - demand over time
plt.figure(figsize=(12, 6))
sns.lineplot(data=inv_opt, x=inv_opt.index, y='Demand')
plt.title('Daily Demand Over Time')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.show()


In [ ]:
# Visualizing the data - inventory over time
plt.figure(figsize=(12, 6))
sns.lineplot(data=inv_opt, x=inv_opt.index, y='Inventory')
plt.title('Daily Inventory Over Time')
plt.xlabel('Date')
plt.ylabel('Inventory')
plt.show()


Insert into ChromaDB

In [ ]:
# Create natural language summaries
def row_to_summary(row, date):
    doi = row['days_of_inventory']
    stock_status = (
        "overstock risk" if doi > 30 else
        "stockout risk" if doi < 5 else
        "healthy stock level"
    )

    return (
        f"On {date.date()}, product {row['Product_ID']} had a demand of "
        f"{row['Demand']} units and an ending inventory of {row['Inventory']} units. "
        f"This corresponds to approximately {doi:.1f} days of inventory, indicating {stock_status}."
    )

In [ ]:
documents = []
metadatas = []
ids = []

for idx, row in inv_opt.iterrows():
    summary = row_to_summary(row, date=idx)
    documents.append(summary)
    metadatas.append({
        "purpose": "case_study",
        "domain": "inventory",
    })
    ids.append(f"{row['Product_ID']}_{idx.strftime('%Y%m%d')}") # adding unique keys

In [ ]:
inventory_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

# inventory_collection.peek()

### More Inventory Case Studies

In [ ]:
documents = []
metadatas = []
ids = []

inventory_casestudy_1 = """
Background
A mid-sized retail company was grappling with persistent overstocking issues. Their warehouses were filled with unsold merchandise, leading to escalating storage costs, high rates of obsolescence, and unnecessary markdowns. The excess inventory tied up working capital and hindered the company’s ability to pivot with market trends.

Solution
The company implemented an inventory optimization strategy centered around automated demand planning. By analyzing historical sales data and seasonality trends, they fine-tuned stock levels to reflect true customer demand. Advanced algorithms generated accurate forecasts, and automated reorder processes helped maintain optimal inventory without human intervention.

Result
Within the first year of implementation, the retailer reported an 18% reduction in total inventory volume. This translated into annual savings of over $500,000 due to decreased warehousing costs and better stock turnover. The company also saw improved agility in responding to market fluctuations, enabling it to introduce new products faster.
"""
documents.append(inventory_casestudy_1)
metadatas.append({
    "purpose": "case_study",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_2 = """
The Challenge
A growing e-commerce business frequently encountered product stockouts, particularly during peak seasons. These shortages frustrated customers, led to missed sales, and damaged brand credibility. Manual tracking of reorder points proved unreliable due to fluctuating demand and inconsistent vendor lead times.

The Optimization Strategy
To address the issue, the company adopted automated reorder point optimization. Using built-in ERP tools, they calculated reorder points based on a combination of sales velocity, supplier lead time, and safety stock requirements. The system monitored inventory in real time and automatically triggered replenishment orders when thresholds were reached.

Outcomes
Stock availability increased by 30%, particularly during high-demand periods. The automation reduced manual errors and enabled the procurement team to focus on supplier relationship management rather than operational fire-fighting. As a result, customer satisfaction and repeat purchase rates noticeably improved.
"""
documents.append(inventory_casestudy_2)
metadatas.append({
    "purpose": "solution",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_3 = """
Context
A manufacturing firm was experiencing a slow inventory turnover ratio, holding onto materials and finished goods for extended periods. This inefficiency led to cash flow constraints and limited the company’s ability to invest in innovation and market expansion.

Inventory Optimization Strategy
The company conducted a full SKU analysis using ABC classification, identifying low-value and slow-moving items that could be phased out or reordered less frequently. Demand forecasting models were implemented to align procurement schedules with actual sales patterns, reducing bulk purchases of underperforming items.

Projected Financial Impact
The improved turnover ratio—from 3x to 6x annually—freed up substantial working capital. Conservative estimates suggested annual cost savings exceeding $300,000 through reduced holding costs, better supplier terms, and fewer product write-offs. Additionally, the faster turnover cycle improved forecasting accuracy and operational efficiency across the board.
"""
documents.append(inventory_casestudy_3)
metadatas.append({
    "purpose": "cost_estimates",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_4 = """
The Problem
A consumer goods distributor experienced frequent delays due to over-reliance on a single supplier network. Disruptions in shipments led to gaps in product availability and increased costs due to last-minute emergency orders from alternate vendors.

Solution Framework
The company shifted towards a multi-sourcing strategy, supported by ERP-based vendor scorecards and delivery tracking. This allowed them to evaluate suppliers based on lead times, reliability, and cost efficiency. Safety stock policies were recalibrated, and predictive models helped anticipate disruptions before they impacted inventory.

Results
The changes improved the company’s order fulfillment rate from 85% to 97% within six months. With improved visibility and supplier accountability, the business not only reduced emergency procurement costs but also boosted customer retention due to enhanced delivery reliability.
"""
documents.append(inventory_casestudy_4)
metadatas.append({
    "purpose": "case_study",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_5 = """
Background
A mid-sized e-commerce retailer faced challenges with inventory inaccuracies, leading to frequent stockouts and overstock situations. These issues resulted in lost sales opportunities and increased holding costs.​

Solution
The company implemented Hopstack's intelligent inventory management solutions, which provided real-time visibility into stock levels across all channels. By leveraging automated inventory allocation and dynamic replenishment strategies, the retailer could maintain optimal stock levels and respond swiftly to demand fluctuations.​

Outcome
Post-implementation, the retailer achieved 99.9% inventory accuracy and a 50% decline in stockouts. Additionally, there was a 35% improvement in order fill rates, leading to enhanced customer satisfaction and reduced operational costs.​
"""
documents.append(inventory_casestudy_5)
metadatas.append({
    "purpose": "case_study",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_6 = """
The Challenge
Traditional forecasting methods, such as moving averages, were insufficient for a consumer electronics distributor dealing with seasonal demand and rapid product obsolescence. This led to either excess inventory or missed sales opportunities.​

The Optimization Strategy
By adopting a machine learning-centric approach, the distributor utilized models that considered various features, including historical sales data, market trends, and promotional activities. This approach allowed for more accurate demand forecasting and inventory optimization.​

Outcomes
The machine learning models provided explainable and actionable insights, enabling the distributor to adjust inventory levels proactively. This led to a significant reduction in excess stock and improved the alignment of inventory with actual demand patterns.​
"""
documents.append(inventory_casestudy_6)
metadatas.append({
    "purpose": "solution",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_7 = """
Context
A fashion retailer struggled with high fulfillment costs due to inefficient inventory placement and frequent stock transfers between warehouses. These inefficiencies increased shipping times and costs.​

Inventory Optimization Strategy
The retailer implemented Hopstack's dynamic inventory allocation system, which automated the distribution of stock based on real-time demand and predefined business rules. This system prioritized high-demand products and optimized stock levels across all locations.​

Projected Financial Impact
By optimizing inventory placement, the retailer reduced inter-warehouse transfers by 40% and decreased average shipping times by 25%. These improvements led to an estimated annual savings of $300,000 in fulfillment costs.
"""
documents.append(inventory_casestudy_7)
metadatas.append({
    "purpose": "cost_estimates",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_8 = """
The Problem
A home goods supplier faced challenges with order fulfillment efficiency, resulting in delayed shipments and customer dissatisfaction. Manual processes were time-consuming and prone to errors.​

Solution Framework
The supplier adopted Hopstack's automated inventory management features, including cycle counting, inventory audits, and automated replenishment. These tools streamlined operations and ensured accurate stock levels.​

Results
The automation led to a 30% reduction in order processing times and a 20% decrease in fulfillment errors. Improved efficiency enhanced customer satisfaction and allowed the supplier to handle increased order volumes without additional staffing.​
"""
documents.append(inventory_casestudy_8)
metadatas.append({
    "purpose": "case_study",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_9 = """
Background: Stanley Black & Decker, a global leader in tools and storage, faced challenges with excessive finished goods inventory, leading to high carrying costs and reduced operational efficiency.​

Solution: By implementing Logility’s Voyager Inventory Optimization™, the company aimed to balance inventory levels with actual demand, ensuring optimal stock availability without overstocking.​

Outcome: The initiative led to a 27% reduction in finished goods inventory, freeing up significant working capital and enhancing the company's ability to respond swiftly to market demands.
"""
documents.append(inventory_casestudy_9)
metadatas.append({
    "purpose": "case_study",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_10 = """
Scenario: A mid-sized retailer maintained high inventory levels to prevent stockouts, resulting in tied-up capital and increased storage costs.​

Strategy: By adopting an inventory optimization solution, the retailer analyzed demand patterns, adjusted safety stock levels, and streamlined replenishment processes.​

Financial Impact: The retailer achieved a 10%–30% reduction in inventory holdings, translating to substantial savings in storage costs and freeing up capital for other strategic investments.
"""
documents.append(inventory_casestudy_10)
metadatas.append({
    "purpose": "cost_estimate",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_11 = """
Challenge: A distributor faced issues with redundant stock and frequent markdowns due to overstocking and misaligned inventory levels.​

Approach: Implementing time-phased inventory planning allowed the distributor to align stock levels with actual demand forecasts, set optimal safety stock levels, and determine precise replenishment quantities for each item.​

Results: The distributor eliminated redundant stock, accelerated inventory turnover, prevented product obsolescence, and reduced markdown quantities, leading to improved profitability and warehouse space utilization.
"""
documents.append(inventory_casestudy_11)
metadatas.append({
    "purpose": "solution",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_12 = """
Problem: A manufacturing company experienced disruptions due to supply chain variability, affecting production schedules and customer satisfaction.​

Solution: By enhancing visibility into procurement, production costs, and service requirements through an inventory optimization system, the company could proactively manage supply chain uncertainties.​

Outcome: The company achieved more efficient operations, reduced the need for expedited production and transportation, minimized disruptions, and improved customer service levels.
"""
documents.append(inventory_casestudy_12)
metadatas.append({
    "purpose": "case_study",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_13 = """
Issue: Frequent stockouts led to delayed deliveries and dissatisfied customers for an online retailer.​

Strategy: Implementing demand alignment and forecasting tools within an inventory optimization solution enabled the retailer to better predict customer needs and maintain appropriate stock levels.​

Impact: The retailer experienced increased customer service levels, more consistent delivery performance, and strengthened brand loyalty. ​
"""
documents.append(inventory_casestudy_13)
metadatas.append({
    "purpose": "solution",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_14 = """
Challenge: Warehouse staff struggled with inefficiencies due to inaccurate inventory data, leading to misplaced items and delayed order fulfillment.​

Implementation: An inventory optimization system provided real-time, accurate inventory information, enabling employees to make informed decisions and streamline operations.​

Result: Employees became more confident and efficient in their roles, leading to shorter lead times, lower operational costs, and improved overall productivity. ​
"""
documents.append(inventory_casestudy_14)
metadatas.append({
    "purpose": "case_study",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_15 = """
Background: Canadian Tire, a major Canadian retailer, faced unprecedented challenges during the early days of the COVID-19 pandemic, with 40% of its brick-and-mortar locations shut down due to quarantine measures.​

Solution: Leveraging ThoughtSpot's self-service business intelligence tools, Canadian Tire quickly identified shifting customer demands and adjusted its inventory accordingly. This agile approach allowed the company to reallocate stock to meet the changing needs of its customers.​

Outcome: Despite significant operational disruptions, Canadian Tire achieved a 20% increase in sales, demonstrating the effectiveness of data-driven inventory management in responding to rapidly changing market conditions.
"""
documents.append(inventory_casestudy_15)
metadatas.append({
    "purpose": "case_study",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_16 = """
Challenge: Businesses often struggle with managing vast inventories, leading to inefficiencies and increased holding costs.​

Approach: Implementing ABC analysis allows companies to categorize inventory into three groups: 'A' items (high value, low quantity), 'B' items (moderate value and quantity), and 'C' items (low value, high quantity). This classification helps prioritize inventory management efforts based on the significance of each category.​

Benefits: By focusing resources on 'A' items, businesses can ensure optimal stock levels for high-value products, reduce excess inventory, and improve overall operational efficiency.
"""
documents.append(inventory_casestudy_16)
metadatas.append({
    "purpose": "solution",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_17 = """
Background: Fabuwood, a cabinet manufacturer, faced challenges in managing its extensive product portfolio, leading to inefficiencies and increased costs.​

Solution: By adopting ThoughtSpot's analytics platform, Fabuwood empowered its team to access and analyze SKU-level data, facilitating informed decisions about product offerings.​

Outcome: The company streamlined its product lineup, eliminating underperforming SKUs, reducing inventory costs, and enhancing profitability.
"""
documents.append(inventory_casestudy_17)
metadatas.append({
    "purpose": "case_study",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

inventory_casestudy_18 = """
Solution: Enhancing Supply Chain Resilience with Business Intelligence
Challenge: Supply chain disruptions can significantly impact inventory levels and customer satisfaction.​

Approach: Implementing business intelligence tools allows companies to analyze factors like seasonal trends, market conditions, and customer behavior, enabling them to optimize inventory levels, production schedules, and distribution strategies.​

Benefits: By proactively managing supply chain variables, businesses can minimize stockouts, reduce excess inventory, and improve overall operational efficiency. ​
"""
documents.append(inventory_casestudy_18)
metadatas.append({
    "purpose": "solution",
    "domain": "inventory",
})
ids.append(str(uuid.uuid4()))

In [ ]:
inventory_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

### --- ProValet's HVAC Industry Case Studies Article ---

In [ ]:
url = "https://www.provalet.io/guides-posts/hvac-industry-case-studies"
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "html.parser")

full_text = ' '.join([p.get_text() for p in soup.find_all('p')])


In [ ]:
# Create a splitter
text_splitter = CharacterTextSplitter(
    separator=" ",
    chunk_size=700,
    chunk_overlap=100,
    length_function=len
)

chunks = text_splitter.split_text(full_text)

# Check one
print(f"Split into {len(chunks)} chunks. First one:\n\n{chunks[0]}")

In [ ]:
# Convert chunks in LangChain documents

docs = [
    Document(
        page_content=chunk,
        metadata={
            "purpose": "case_study",
            "domain": "energy"
        },
    )
    for chunk in chunks
]


Insert into ChromaDB

In [ ]:
# extract and format for ChromaDB
documents = [doc.page_content for doc in docs]
metadatas = [doc.metadata for doc in docs]
ids = [str(uuid.uuid4()) for _ in docs]

In [ ]:
energy_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)
# energy_collection.peek()

### --- Summary of Economics ROI Impact Evaluation Studies ---

In [ ]:
# Fetching the PDF
url = "https://www.energy.gov/sites/default/files/2025-01/eere-iso-roi-report-2024.pdf"
response = requests.get(url)

with open("eere_roi_report.pdf", "wb") as f:
    f.write(response.content)


In [ ]:
# Parse text and tables
tables = []
paragraphs = []

with pdfplumber.open("eere_roi_report.pdf") as pdf:
    for page_num, page in enumerate(pdf.pages):
        # Extract raw text
        if page_num == 7: # don't extract the references page
          break

        text = page.extract_text()
        if text:
            paragraphs.append({
                "page": page_num + 1,
                "text": text
            })

        # Extract tables
        page_tables = page.extract_tables()
        for table in page_tables:
            df = pd.DataFrame(table[1:], columns=table[0])
            tables.append({
                "page": page_num + 1,
                "table": df
            })

cleaned_tables = []

for table_obj in tables:
    df = table_obj["table"]

    # Clean column names
    df.columns = [col.replace('\n', ' ').strip() for col in df.columns]

    # Clean every cell in the DataFrame
    cleaned_df = df.map(lambda cell: cell.replace('\n', ' ').strip() if isinstance(cell, str) else cell)

    # Store cleaned version
    cleaned_tables.append({
        "page": table_obj["page"],
        "table": cleaned_df
    })

In [ ]:
# Show output
for p in paragraphs:
    print(f"\n--- Page {p['page']} ---")
    print(p["text"])

for i, t in enumerate(cleaned_tables):
    print(f"\n--- Table from Page {t['page']} ---")
    display(t["table"])

print(f"{len(paragraphs)} paragraphs extracted.")
print(f"{len(tables)} tables extracted.")

In [ ]:
# Pre-process Table 1, page 5 for ChormaDB
documents = []
metadatas = []
ids = []

gross_table = cleaned_tables[2]["table"]

for _, row in gross_table.iterrows():
    if "Sum Totals" in str(row[1]):
        continue

    program_full = str(row[0]).strip()
    parts = program_full.split()
    program_code = parts[0]
    program_year = parts[1] if len(parts) > 1 else "unknown"

    investment = row[3]
    benefit = row[5]

    summary = (
        f"In {program_year}, the DOE’s {program_code} program invested approximately {investment} million USD "
        f"and achieved a gross economic benefit of {benefit} million USD. "
        "These values are adjusted for inflation to 2024 and are not discounted."
    )

    metadata = {
        "purpose": "cost_estimates",
        "domain": "energy",
    }

    documents.append(summary)
    metadatas.append(metadata)
    ids.append(str(uuid.uuid4()))

# Store Table 1 Data
energy_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)


In [ ]:
# Pre-process Table 2, page 5 for ChormaDB
documents = []
metadatas = []
ids = []

roi_table = cleaned_tables[3]["table"]

for _, row in roi_table.iterrows():
    if "Totals" in str(row[0]):
        continue

    program_full = str(row[0]).strip()
    parts = program_full.split()
    program_code = parts[0]
    program_year = parts[1] if len(parts) > 1 else "unknown"

    investment = row[1]
    gross_benefit = row[2]
    net_benefit = row[3]
    npv_3 = row[4]
    npv_7 = row[5]
    ratio_3 = row[6]
    ratio_7 = row[7]

    summary = (
        f"In {program_year}, the DOE’s {program_code} program invested {investment} million USD. "
        f"This generated {gross_benefit} million USD in total benefits and {net_benefit} million USD in net economic gains "
        "(all values inflation-adjusted to 2024 and not discounted). "
        f"The net present value (NPV) was {npv_3}M at a 3% discount rate and {npv_7}M at 7%. "
        f"The corresponding benefit-to-cost ratios were {ratio_3} and {ratio_7}."
    )

    metadata = {
        "purpose": "cost_estimates",
        "domain": "energy",
    }

    documents.append(summary)
    metadatas.append(metadata)
    ids.append(str(uuid.uuid4()))

# Store Table 2 Data
energy_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)


In [ ]:
# Chunking and pre-processing paragraphs
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=700,
    chunk_overlap=100,
    length_function=len
)

documents = []
metadatas = []
ids = []

for para in paragraphs:
    page = para["page"]
    chunks = text_splitter.split_text(para["text"])

    for chunk in chunks:
        documents.append(chunk)
        metadatas.append({
            "purpose": "case_study",
            "domain": "energy"
        })
        ids.append(str(uuid.uuid4()))

# Store paragraph chunks
energy_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

### --- Energy Efficient Cities Data ---

In [ ]:
# https://www.esmap.org/node/231

Building & Heating

In [ ]:
energy_cases = []
energy_metadata = []

lviv_casestudy = """
The Ukrainian city of Lviv was able to reduce annual energy consumption in its public buildings by about 10 percent and tap water consumption by about 12 percent through a Monitoring and Targeting (M&T) program to control energy and water consumption. This generated an estimated net savings of 9.5 million UAH (US$1.2 million) as of 2010. The M&T program was launched in December 2006 and became fully operational by May 2007. It provided the city management with monthly consumption data for district heating, natural gas, electricity and water in all of the city’s 530 public buildings.  Under the program, utility use is reported and analyzed monthly; targets for monthly utility consumption are determined annually based on historical consumption and negotiations on an adjustment (in cases of foreseeable changes in consumption patterns). Actual consumption is reviewed monthly against the target, with deviations spotted and acted upon immediately and the performance of buildings is communicated to the public through a display campaign.

The M&T program achieved significant savings with minimal investment and recurring program costs. These utility bill reductions have been valuable in light of fiscal constraints and increasing energy prices. The program benefited from a crucial initial condition where most of the city’s public buildings were already metered for energy and water consumption and that the city had been collaborating with international aid programs in municipal energy since the late 1990s.

Strong city government leadership and commitment were key success factors of Lviv’s public buildings energy and water M&T program. A new Energy Management Unit (EMU) was established within the city administration and resources were mobilized to train all personnel with line responsibility on building utility use in an administrative division, unit, or building. The M&T system established responsibility, created transparency, and enabled informed control of energy and water use in public buildings, laying a solid foundation for sustained improvements in energy and water efficiency.

"""
lviv_metadata = {
  "use_case": "public building energy and water overuse",
  "solution_type": "monitoring and targeting system",
  "impact": "reduced energy consumption by 10% and water consumption by 12%",
  "estimated_savings": "9.5 million UAH (US$1.2 million) by 2010",
  "region": "Lviv, Ukraine",
  "business_type": "municipal public buildings"
}

kyiv_casestudy = """
The Kiev Public Buildings Energy Efficiency Project (2000–2005) retrofitted 1,270 public buildings—schools, hospitals, and cultural institutions—with modern energy efficiency systems. These included heat meters, weather-stripping, radiator reflectors, and automated substations. The initiative achieved a 26% reduction in heating energy use (333,423 Gcal/year) and helped avoid US$31.7 million in heating bills by 2011.

Funded with US$27.4 million from the World Bank, Swedish Government, and local administration, the project spurred development of a domestic energy efficiency industry and heightened public awareness. Key innovations included aligning tariff reforms with retrofits, ensuring operation and maintenance through competitive contracting, and simplifying project scope to reduce risk. The effort proved financially sustainable, with an internal rate of return of 26.6% and strong replication potential across Ukraine.
"""
kyiv_metadata = {
  "use_case": "inefficient heating in public buildings",
  "solution_type": "retrofitting with demand and supply-side measures",
  "impact": "26% reduction in heating energy use (333,423 Gcal/year)",
  "estimated_savings": "US$31.7 million in heating bills (2005–2011)",
  "region": "Kiev, Ukraine",
  "business_type": "municipal public buildings"
}

munster_casestudy = """
By mandating low-energy building standards in sales contracts of city-owned land, the City of Münster (Germany) caused a market transformation that led to 80 percent of all new buildings constructed in 2010, even those not built on city-owned land, to follow the city’s energy efficiency requirements. The standard, locally known as Niedrig Energie Haus (NEH), imposes stringent thermal performance requirements for the building envelope which exceed the existing German federal building regulations by 30 percent.

The implementation of NEH led to the construction of over 5,600 low-energy housing units and 85 more energy efficient commercial buildings between 1997 and 2010, saving the city 13 million kWh per year while reducing annual CO2 emissions by at least 3,400 tons in the residential sector by 2010. The benefit-cost ratio to the city was over 6 to 1 and the estimated incremental cost to homeowners was, on average, about €2,600 per house (about 1.4 percent of construction costs). The transformative outcome and positive market reactions encouraged the City Council to propose a passive house standard in the fall of 2010.

The program was fairly low in cost, less than about a half million Euros, invested by the city to achieve substantial savings. However, some of the conditions were very unique; the city owned about 50 percent of the undeveloped land, city residents were supportive of broad scale climate measures, and homeowners had the financial means to absorb a modest incremental cost for the low-energy houses.
"""
munster_metadata = {
  "use_case": "energy inefficiency in new residential and commercial buildings",
  "solution_type": "mandated low-energy building standards via land sales contracts",
  "impact": "saved 13 million kWh/year and reduced CO2 emissions by 3,400 tons/year",
  "estimated_savings": "benefit-cost ratio over 6:1; average homeowner cost €2,600 (~1.4% of construction cost)",
  "region": "Münster, Germany",
  "business_type": "residential and commercial construction"
}

tianjin_casestudy = """
Tianjin is one of the most successful Chinese cities in compliance enforcement of building energy efficiency codes (BEECs). Results of recent annual national inspections organized by the Ministry of Housing and Urban and Rural Development (MoHURD) indicate that compliance of BEECs in new residential and commercial buildings in Tianjin is close to 100 percent, compared to the 80 percent average across nearly three dozen large cities inspected by MoHURD in 2008. More remarkable is the fact that, in terms of building envelope thermal integrity, the currently enforced residential BEEC in Tianjin (identified as DB29-1-2007) is 30 percent more stringent than what is required by the pertinent national BEEC (identified as JGJ 26-95).

In 1997, Tianjin introduced its first mandatory residential BEEC (identified as DB29-1-97), which is equivalent to the requirements of the JGJ 26-95, the national model BEEC for cold climate regions enacted in 1995. DB29-1-97 was enforced from 1998 to 2004. Enforcement actually began on January 1, 2005; it was based on an earlier version which was updated and reenacted on June 1, 2007. This case study covered five years of enforcement of DB29-1-2007, from 2005 to 2009.

From 2005 to 2009, about 70 million m2 of new residential buildings were completed in the urban areas of Tianjin. Many were subject to compliance of DB29-1-97 because their construction permits were issued before January 1, 2005. About 50 million m2 of them were designed according to Tianjin’s DB29-1-2007. This puts the upper range (assuming full construction compliance) of the calculated space heating energy savings of these new buildings at about 870 GWh/year, compared with the baseline of complying with DB29-1-97 (or the national JGJ 26-95). Heat energy saved would be sufficient to provide winter space heating for about 200,000 apartments (at 100 m2 each) built to comply with DB29-1-2007. Compared with the baseline, the estimated simple payback period for compliance with DB29-1-2007 based on avoided cost of heating service is less than seven years, attractive even if the energy efficiency (EE) measures have a lifespan of 15 years.

Tianjin’s efforts to go beyond national BEEC requirements marked a departure from the mostly central government-driven BEEC regulation of the past in China. Tianjin began piloting residential BEEC in the late 1980s, despite the fact that it has taken about 15 years for Tianjin to achieve a high degree of compliance. Tianjin has demonstrated the importance of the following factors in achieving BEEC compliance: (i) a well-established building construction management system, (ii) standardized and structured procedures for compliance enforcement, (iii) broad-based capacity of the construction trades to meet compliance requirements, including technical skills and availability of parts and materials, (iv) consumers’ ability and willingness to pay for the costs of BEEC compliance, and (v) local government resources, support, and commitment to implementing increasingly stringent BEECs.
"""
tianjin_metadata = {
  "use_case": "inefficient thermal performance in new residential buildings",
  "solution_type": "strict enforcement of advanced building energy efficiency codes",
  "impact": "saved up to 870 GWh/year in heating energy; sufficient to heat 200,000 apartments",
  "estimated_savings": "simple payback period under 7 years for DB29-1-2007 compliance",
  "region": "Tianjin, China",
  "business_type": "residential construction and real estate development"
}

energy_cases.append(lviv_casestudy)
energy_cases.append(kyiv_casestudy)
energy_cases.append(munster_casestudy)
energy_cases.append(tianjin_casestudy)

energy_metadata.append(lviv_metadata)
energy_metadata.append(kyiv_metadata)
energy_metadata.append(munster_metadata)
energy_metadata.append(tianjin_metadata)

Lighting

In [ ]:
akola_casestudy = """
More than 11,500 street lights (standard fluorescent, mercury vapor, sodium vapor) were
replaced with efficient, T5 fluorescent tube lamps as part of the energy efficient street
lighting project by Akola Municipal Corporation (AMC), an Indian Urban Local Body
(ULB) in the state of Maharashtra. The project used an energy savings performance
contracting approach under which the contractor, Asia Electronics Limited (AEL),
financed all investment costs, implemented the project, maintained the newly-installed
lamps, and received a portion of the energy savings to recover its investment. Through
the project, AMC was able to improve the service level of street lighting with lower costs
and lower electricity bills.

The project resulted in annual energy savings of 2.1 million kWh (56%), representing
reduced electric bills for the city totaling INR (Indian Rupee) 6.4 million (US$133,000)
per year. Since the total project cost was only about INR5.7 million (US$120,000), the
project payback period was less than 11 months. Because the project cost was entirely
financed by AEL, acting as an Energy Service Company (ESCO), AMC did not have to
make any upfront investment or assume performance risks under the project. Under the
energy savings performance contract (ESPC), compensation to AEL was based on a
shared savings approach under which AMC paid AEL 95% of the energy bill savings
over the contract’s 6-year duration. AEL was also paid an annual fixed fee for
maintaining the lamps and fixtures.

The project’s success has already led to similar projects being implemented in the states
of Maharashtra and Madhya Pradesh. AEL, the World Bank, and Government of India
are also pursuing options to mobilize carbon financing under the Clean Development
Mechanism (CDM) to further enhance benefits to municipalities from such projects (e.g.,
Pune, Latur).
"""
akola_metadata = {
  "use_case": "high electricity costs from inefficient street lighting",
  "solution_type": "energy-efficient street lighting using performance-based ESCO model",
  "impact": "56% energy savings; 2.1 million kWh/year reduction",
  "estimated_savings": "INR 6.4 million (US$133,000) annually; project payback < 11 months",
  "region": "Akola, Maharashtra, India",
  "business_type": "urban municipal street lighting"
}

portland_casestudy = """
In 2001, the City of Portland replaced most of its incandescent traffic signal lights with
new, highly efficient light-emitting diodes (LEDs). Using an innovative leasing
arrangement to amortize the investment costs, Portland’s Signal and Street Lighting
Division was able to replace 13,382 red and green existing incandescent lamps with
LEDS. The project resulted in 4.9 million kilowatt-hours (kWh) of annual savings in
energy consumption (representing an 80% reduction in energy usage), and a reduction of
about 2,880 tons of annual CO2 emissions. While the project benefitted from a special
tax credit and utility rebate, its economics were favorable regardless – the total
investment of $2.2 million (before incentives) resulted in annual utility bill and
maintenance savings of $380,000 per year, or a payback of 5.8 years (down to 3.1 years
with the incentives). However, since the lamps were leased to the City, no upfront
investment was required. The energy savings also helped the City avoid budgetary
repercussions from an unexpected 40% electric rate hike from the power utility.
"""
portland_metadata = {
  "use_case": "high energy use and maintenance costs from incandescent traffic signals",
  "solution_type": "LED replacement using financial lease model",
  "impact": "80% energy reduction; 4.9 million kWh/year saved; 2,880 tons CO2 reduction annually",
  "estimated_savings": "US$380,000/year; payback in 5.8 years (3.1 with incentives)",
  "region": "Portland, Oregon, USA",
  "business_type": "municipal traffic signal infrastructure"
}

losangeles_casestudy = """
The City of Los Angeles (LA) Light Emitting Diode (LED) street lighting project is the
largest LED street lighting retrofit ever undertaken globally—a collaboration between the LA
Bureau of Street Lighting, the LA Mayor’s Office, the LA Department of Water & Power,
and the Clinton Climate Initiative (CCI) Cities Program. Over a period of five years (2009-
2014), the project will replace 140,000 of the city’s more than 209,000 street lights with LED
technology which is expected to enhance the quality of municipal street lighting, reduce light
pollution, improve street safety, and save energy and money. The US$56.9 million
investment required will provide an estimated US$10 million in annual energy and
maintenance cost savings (68.6 GWh/year) while avoiding at least 40,500 tons of CO2 e
emissions each year.
The project was initiated only after comprehensive planning and evaluation yielded positive
results and feedback from stakeholders, a constructive model for other cities to follow. The
technology and financial evaluation of the project was conducted with the help of CCI and
other external entities. The Bureau financed the project with a seven-year, US$40 million
dollar loan from the city, along with a US$3.6 million contribution from the Bureau’s Street
Lighting Maintenance Assessment Fund (SLMAF) and a rebate from the LA Department of
Water & Power-totaling US$13.2 million. The loan amount will be repaid to the city from
energy and maintenance cost savings. Other benefits include the elimination of hazardous
mercury from street lighting fixtures, job creation, improved illumination and visual quality
of street lighting, enhanced safety on the roads, and reduced local and global air pollution.
The positive feedback this project has received from local communities, politicians, law
enforcement officials and others strengthens the case for the use of LEDs in municipal
lighting projects.
"""
losangeles_metadata = {
  "use_case": "high energy and maintenance costs from traditional street lighting",
  "solution_type": "large-scale LED street lighting retrofit",
  "impact": "68.6 GWh/year saved; avoided 40,500 tons CO2e emissions annually",
  "estimated_savings": "US$10 million/year with US$56.9 million investment; 7-year loan repayment from savings",
  "region": "Los Angeles, California, USA",
  "business_type": "municipal street lighting infrastructure"
}

energy_cases.append(akola_casestudy)
energy_cases.append(portland_casestudy)
energy_cases.append(losangeles_casestudy)

energy_metadata.append(akola_metadata)
energy_metadata.append(portland_metadata)
energy_metadata.append(losangeles_metadata)

In [ ]:
# energy_case_metadata = []
# energy_case_id = []
# for energy_case in energy_cases:
#   energy_case_metadata.append({
#         "purpose": "case_study",
#         "domain": "inventory",
#   })
#   energy_case_id.append(str(uuid.uuid4()))

# energy_collection.add(
#   documents=energy_cases,
#   metadatas=energy_case_metadata,
#   ids=[energy_case_id]


### --- US DOE Data ---

In [ ]:
# https://www.energy.gov/eere/iedo/case-studies-system

Plant-Wide

In [ ]:
alcoa_casestudy = """
Alcoa, the largest aluminum manufacturer in the U.S., is one of the most energy-intensive industrial operations in North America, with energy costs representing a substantial share of both alumina refining and aluminum production expenses. In response to rising energy prices, Alcoa embedded energy efficiency into its strategic and sustainability goals, initiating formal efforts as early as 1990. The company's first sustainability targets were published in 1993 and have been updated regularly since. As of 2009, Alcoa set ambitious energy intensity reduction goals—10% for primary operations and 20% for other businesses by 2020, with further reductions targeted for 2030.

To meet these goals, Alcoa launched several internal programs, including the Energy Efficiency Network, which conducts on-site energy audits and identifies cost-saving measures. To date, this program has uncovered more than $100 million in potential savings.

A key element in Alcoa’s success has been the active involvement of C-suite executives, particularly the CEO and the President of Energy, Rick Bowen. Their engagement not only elevated the profile of energy management across the enterprise but also led to the integration of energy performance into leadership compensation structures, boosting accountability and motivating staff.

In 2010, Alcoa formalized its energy commitment by joining the U.S. Department of Energy’s Save Energy Now LEADER initiative, pledging to reduce energy consumption by 25% over 10 years. Although this target exceeded Alcoa’s internal goals, the LEADER program’s tools, training, and collaborative benefits convinced the corporate leadership to support the initiative.

Financing for energy projects is coordinated through a centralized capital budget, where each business unit competes by presenting high-ROI energy-saving projects. Preference is given to projects with a payback period under two years. The President of Energy plays a pivotal role in developing these proposals and guiding their presentation to senior executives.

Alcoa’s approach focuses not only on capital-intensive measures but also on behavior-based and low-cost improvements, including staff training, efficient production scheduling, and broader awareness campaigns. The company’s energy leadership team—Rick Bowen, Richard Notte (VP of Energy Services), and Walt Brockway (Manager of Global Energy Efficiency)—collaborate closely with public relations to celebrate milestones and foster a culture of sustainability.
Alcoa’s partnership with the DOE has also proven beneficial, opening up access to diagnostic tools, peer networks, and successful case studies across industries. Alcoa credits this partnership for catalyzing internal conversations and encouraging cross-industry collaboration.

In conclusion, Alcoa’s integrated energy strategy—driven by strong executive leadership, robust internal programs, and external partnerships—has transformed energy efficiency into a core business advantage. By emphasizing both immediate financial returns and long-term sustainability, the company has built a resilient and replicable model for industrial energy management.
"""
alcoa_metadata = {
  "use_case": "high energy intensity and cost in industrial aluminum manufacturing",
  "solution_type": "enterprise-wide energy efficiency program with C-suite accountability and DOE partnership",
  "impact": "identified over $100 million in savings; committed to 25% reduction in energy intensity in 10 years",
  "estimated_savings": "up to $500,000 in one plant alone; multiple facilities seeing substantial operational savings",
  "region": "United States",
  "business_type": "industrial aluminum manufacturing and refining"
}

chrome_deposit_corporation_casestudy = """
Chrome Deposit Corporation (CDC), a metal finishing company specializing in hard chrome plating and electro-discharge texturing, operates a facility in Newark, Delaware. Recognizing the energy-intensive nature of its operations, CDC partnered with the Delaware Manufacturing Extension Partnership (DEMEP), the University of Delaware’s Center for Energy and Environmental Policy, and the U.S. Department of Energy’s Industrial Technologies Program (ITP) to conduct an Energy Savings Assessment.​
The assessment identified several opportunities for energy efficiency improvements. By implementing inexpensive and straightforward changes to infrastructure and operational processes, CDC achieved a 25% reduction in energy costs per unit. These measures not only minimized the company's environmental impact but also enhanced its energy management practices, all while allowing for continued operational growth.​
Key initiatives included optimizing equipment usage, improving process controls, and fostering a culture of energy awareness among employees. The success of these efforts underscores the potential for small to medium-sized manufacturers to achieve significant energy savings through targeted assessments and collaborative partnerships.
"""
chrome_deposit_corporation_metadata = {
  "use_case": "high energy consumption in metal finishing operations",
  "solution_type": "energy savings assessment and implementation of efficiency measures",
  "impact": "25% reduction in energy costs per unit; enhanced energy management practices",
  "estimated_savings": "significant annual cost savings with minimal investment",
  "region": "Newark, Delaware, USA",
  "business_type": "metal finishing and chrome plating"
}

ccp_composites_casestudy = """
CCP Composites, a global leader in the production of gel coats and polyester resins, operates a synthetic resin manufacturing plant in Houston, Texas. Recognizing the escalating energy costs—which had more than doubled between 1998 and 2005 and accounted for about 20% of the plant’s operating budget by 2008—the company sought to improve energy efficiency.​

In collaboration with the U.S. Department of Energy’s Industrial Technologies Program (ITP), CCP Composites implemented an energy management system conforming to ANSI/MSE 2000:2008 standards and SEP requirements. This system was integrated into the company's existing ISO 9001 (quality) and ISO 14001 (environmental) management frameworks.​

Key actions included:​
    Developing an energy management policy.​
    Setting energy performance improvement objectives.​
    Establishing an energy profile and baseline for the Houston site.​
    Conducting self-audits and management reviews prior to third-party certification audits.​

The major energy-consuming systems at the Houston plant included three hot oil boilers, a steam boiler, a thermal oxidizer, a compressed air plant, a chiller, and cooling water pumps.​

As a result of these initiatives, CCP Composites achieved a 14% improvement in energy performance over two years, earning the SEP Gold certification. This accomplishment not only reduced energy costs but also established a proactive framework for ongoing energy management and performance improvement.​
"""
ccp_composites_metadata = {
  "use_case": "rising energy costs in synthetic resin manufacturing",
  "solution_type": "implementation of an energy management system conforming to ANSI/MSE 2000:2008 and SEP requirements",
  "impact": "14% improvement in energy performance over two years; SEP Gold certification achieved",
  "estimated_savings": "significant reduction in energy costs, which had previously doubled between 1998 and 2005",
  "region": "Houston, Texas, USA",
  "business_type": "synthetic resin manufacturing for composites and coatings industry"
}

eck_industries_casestudy = """
Eck Industries, Inc., an aluminum foundry located in Manitowoc, Wisconsin, undertook a comprehensive lighting retrofit to enhance energy efficiency within its 200,000-square-foot manufacturing facility. Collaborating with Wisconsin's Focus on Energy program, the company replaced approximately 360 high-pressure sodium (HPS) fixtures with energy-efficient T8 fluorescent bulbs.​

This initiative led to a 46% reduction in the facility's lighting energy intensity. The project yielded annual operating savings exceeding $55,581 and achieved a payback period of approximately 8 months. To match these savings through increased sales, the company would have needed to generate an additional $1.39 million in revenue, considering a 4% pre-tax profit margin typical in the foundry industry.​
NC Clean Energy Technology Center+3Digital Home Page+3ACerS Bulletin+3

Beyond cost savings, the retrofit improved lighting quality, enhancing the working environment for employees. The success of this project underscores the potential benefits of energy efficiency initiatives in manufacturing settings, particularly when supported by state programs offering technical and financial assistance.
"""
eck_industries_metadata = {
  "use_case": "high energy consumption from outdated lighting systems in manufacturing facility",
  "solution_type": "replacement of 360 high-pressure sodium fixtures with T8 fluorescent bulbs",
  "impact": "46% reduction in lighting energy intensity; annual savings over $55,581; improved lighting quality",
  "estimated_savings": "project payback period of approximately 8 months; equivalent to $1.39 million in additional sales to match savings",
  "region": "Manitowoc, Wisconsin, USA",
  "business_type": "aluminum foundry manufacturing"
}

flambeau_river_casestudy = """
Flambeau River Papers, a historic paper mill in Park Falls, Wisconsin, faced closure in 2006 due to escalating energy costs and outdated equipment. Under new ownership, the mill implemented a comprehensive energy management strategy focusing on efficiency improvements and a transition to renewable energy sources.​

Key Initiatives and Investments:
    Invested $15 million in energy efficiency and alternative energy projects.​
    Received $10 million in loans and matching funds from the U.S. Department of Energy and the state of Wisconsin.​
    Collaborated with Focus on Energy and CleanTech Partners for technical support.​

Energy Efficiency Measures:
    Implemented hot water heat recovery systems and pipe rerouting.​
    Upgraded steam systems and installed accumulator savings devices.​
    Planned to eliminate coal usage by 2009, replacing it with biomass fuels such as pulp, bark, and logging residue.​

Outcomes:
    Achieved $10.5 million in cost savings over 3.5 years.​
    Reduced fossil fuel energy consumption by 100% between 2007 and 2013 (projected).​
    Decreased CO₂ emissions from 1 ton to 620 pounds per ton of paper produced.​
    Increased paper production by 11.9% since 2006.​

This transformation not only revitalized the mill but also contributed to the economic recovery of Park Falls, demonstrating the impact of strategic energy management in industrial settings.​
"""
flambeau_river_metadata = {
  "use_case": "high energy costs and outdated equipment leading to mill closure",
  "solution_type": "comprehensive energy management strategy with efficiency improvements and renewable energy transition",
  "impact": "achieved $10.5 million in cost savings; reduced CO₂ emissions by 69%; increased production by 11.9%",
  "estimated_savings": "$2.6 million in annual energy savings; payback period of 5.25 years",
  "region": "Park Falls, Wisconsin, USA",
  "business_type": "paper manufacturing"
}

freescale_semiconductor_casestudy = """
Freescale Semiconductor's Oak Hill Fab plant in Austin, Texas, successfully implemented an energy management system in accordance with ANSI/MSE 2000:2008 standards. This initiative was part of the U.S. Department of Energy's Industrial Technologies Program and aimed to enhance energy efficiency and reduce operational costs.​
The Department of Energy's Energy.gov+3Energy Efficiency and Renewable Energy+3Energy Efficiency and Renewable Energy+3

Key Achievements:
    Energy Consumption Reduction: Between 2006 and 2009, the plant reduced its annual energy consumption by 28 million kilowatt-hours (kWh) of electricity and 26,000 million British thermal units (Btu) of natural gas.​
    Cost Savings: These reductions translated into annual savings exceeding $2 million.​
    Certification: The Oak Hill plant achieved Silver-level certification under the Superior Energy Performance (SEP) program, demonstrating its commitment to continuous energy performance improvement.​

Implementation Strategy:
    Integration with Existing Systems: The energy management system was integrated into the plant's existing Environmental, Health, and Safety Management System (EHSMS), which is based on ISO 14001 and OHSAS 18001 standards.​
    Team Formation: A core engineering team comprising facilities, environmental, health, and safety staff was established to oversee the implementation.​
    Energy Profile Development: An energy profile for the Oak Hill site was developed, and energy performance indicators were established to monitor progress.​
    Training and Auditing: Staff received training on energy management practices, and the plant conducted self-audits and management reviews prior to third-party certification audits.
"""
freescale_semiconductor_metadata = {
  "use_case": "high energy consumption in semiconductor manufacturing",
  "solution_type": "implementation of an energy management system conforming to ANSI/MSE 2000:2008 and SEP requirements",
  "impact": "reduced annual energy consumption by 28 million kWh of electricity and 26,000 million Btu of natural gas; annual cost savings exceeding $2 million",
  "estimated_savings": "over $2 million annually",
  "region": "Austin, Texas, USA",
  "business_type": "semiconductor manufacturing"
}

harbec_plastics_casestudy = """
HARBEC Inc., a specialty plastics manufacturer in Ontario, New York, implemented a Combined Heat and Power (CHP) system and an energy management system conforming to ISO 50001 standards, achieving significant energy performance improvements and cost savings.​
The Department of Energy's Energy.gov

Key Initiatives and Achievements:
    CHP System Implementation: Installed a CHP system comprising 25 microturbines with a total capacity of 750 kW and five heat exchangers. The system utilizes natural gas to generate electricity and captures waste heat for heating and cooling the facility via an absorption chiller. ​
    Renewable Energy Integration: Complemented the CHP system with two onsite wind turbines (250 kW and 850 kW capacities), enabling the facility to generate over 75% of its electricity needs on-site. ​
    Energy Management System (EnMS): Developed and implemented an EnMS aligned with ISO 50001, integrating it into the company's operations to monitor and improve energy performance continuously. ​
    Superior Energy Performance (SEP) Certification: Achieved SEP Platinum certification by improving energy performance by 16.5% over three years, verified by a third-party audit. ​

Financial Impact:
    Investment: Total cost for implementing SEP and ISO 50001 was $127,000, covering EnMS development, technical assistance, monitoring equipment, and third-party audits. ​
    The Department of Energy's Energy.gov+1The Department of Energy's Energy.gov+1
    Annual Savings: Realized annual energy cost savings of $52,000 through operational improvements without capital investments. ​
    The Department of Energy's Energy.gov+2The Department of Energy's Energy.gov+2The Department of Energy's Energy.gov+2
    Payback Period: Achieved a payback period of 2.4 years for the SEP and ISO 50001 implementation costs. ​
    The Department of Energy's Energy.gov+2The Department of Energy's Energy.gov+2The Department of Energy's Energy.gov+2

Strategic Approach:
HARBEC adopts an "eco-economics" philosophy, evaluating energy projects based on their ability to generate net positive cash flow over their lifetime rather than traditional payback periods. This approach has facilitated investments in energy efficiency and renewable energy projects, contributing to the company's goal of becoming carbon-neutral. ​
"""
harbec_plastics_metadata = {
  "use_case": "high energy costs and environmental impact in plastics manufacturing",
  "solution_type": "implementation of CHP system and ISO 50001-compliant energy management system",
  "impact": "16.5% improvement in energy performance over three years; SEP Platinum certification; over 75% of electricity generated on-site",
  "estimated_savings": "$52,000 in annual energy cost savings; 2.4-year payback period on $127,000 investment",
  "region": "Ontario, New York, USA",
  "business_type": "specialty plastics manufacturing"
}

ingersoll_rand_casestudy = """
Ingersoll Rand, a global industrial firm, implemented a comprehensive three-tiered internal energy audit program in 2005 to enhance energy efficiency across its manufacturing sites. This initiative aimed to reduce global energy intensity by 25% by 2019 and has led to significant cost savings and operational improvements.​

Key Achievements:
    Cost Savings: Since the inception of the program, Ingersoll Rand has saved over $4 million in energy costs.​
    Audit Implementation: The program has been conducted at more than 75% of the company's manufacturing sites globally.​

Three-Tiered Audit Structure:
    Level 1 – Treasure Hunt:
        Focuses on identifying low-cost energy-saving opportunities.
        Engages site-based employee volunteers.
        Targets behavioral changes and simple projects like fixing air leaks and adjusting temperature set-points.​
    Level 2 – Expanded Energy Audit:
        Involves a three-day site visit by a team of energy and engineering experts.
        Examines energy-using equipment and processes.
        Identifies projects suitable for Six Sigma or kaizen initiatives.​
    Level 3 – Investment-Grade Audit:
        Conducted by external consultants for large capital projects.
        Provides detailed financial analysis and project implementation plans.​

Strategic Partnerships and Support:
    Collaborated with the U.S. Department of Energy's Industrial Technologies Program (ITP) and became a Save Energy Now LEADER in 2009.​
    Received assessments from Industrial Assessment Centers (IACs), including those at the Georgia Institute of Technology.​
    Worked with Oak Ridge National Laboratory (ORNL) to develop auditing tools tailored for specific systems, enhancing the capabilities of smaller sites to manage energy use.​

Implementation Strategy:
    Emphasized the business case for energy efficiency projects by evaluating capital costs, return on investment, and technical considerations.​
    Utilized financial incentives from utilities and other sources to support projects with longer payback periods.​
    Addressed challenges such as the invisibility of energy consumption and the need for corporate support by fostering a company-wide philosophy centered on energy efficiency, comfort, and safety.
"""
ingersoll_rand_metadata = {
  "use_case": "reducing energy consumption and costs across global manufacturing sites",
  "solution_type": "implementation of a three-tiered internal energy audit program",
  "impact": "saved over $4 million in energy costs; audits conducted at more than 75% of manufacturing sites",
  "estimated_savings": "over $4 million since 2005",
  "region": "global operations",
  "business_type": "industrial manufacturing"
}

minsouth_metalllurgical_casestudy = """
Mid-South Metallurgical, a commercial heat treating company in Murfreesboro, Tennessee, implemented several energy efficiency upgrades after an assessment by the U.S. Department of Energy’s Industrial Assessment Center at Tennessee Technological University. The facility operates 24 hours a day, 5.5 days a week, across three buildings totaling 21,000 square feet.

Key improvements included:
- Installing new furnace insulation.
- Implementing an electrical demand management system.
- Upgrading furnace burner tubes with energy-efficient equipment.
- Replacing 44 outdated 400-watt metal halide fixtures with 88 new 8-foot fixtures using F32T8 fluorescent bulbs.

Results achieved:
- Overall energy use reduced by 22%.
- Peak demand decreased by 21%.
- Total energy costs lowered by 18%.
- Annual electricity cost savings of approximately $53,000 from furnace insulation.
- Natural gas usage reduced by 20% due to improved burner tubes.
- Lighting operation and maintenance costs reduced by 52%.

These measures not only enhanced energy efficiency but also contributed to significant cost savings for the company.
"""
minsouth_metalllurgical_metadata = {
  "use_case": "high energy costs in heat treating operations",
  "solution_type": "furnace insulation, electrical demand management, burner tube upgrades, lighting retrofit",
  "impact": "22% reduction in energy use, 21% reduction in peak demand, 18% reduction in energy costs",
  "estimated_savings": "$53,000 annual electricity savings from furnace insulation; 20% reduction in natural gas usage; 52% reduction in lighting operation and maintenance costs",
  "region": "Murfreesboro, Tennessee, USA",
  "business_type": "commercial heat treating"
}

nissan_casestudy = """
Nissan Motor Company, a global automotive manufacturer, implemented a comprehensive energy management program across its three U.S. manufacturing facilities located in Smyrna and Decherd, Tennessee, and Canton, Mississippi. Through partnerships with the U.S. Department of Energy's Industrial Technologies Program (ITP) and the Environmental Protection Agency's ENERGY STAR® program, Nissan focused on instilling a company-wide culture of energy efficiency.

Key initiatives included:
- Upgrading and replacing chillers.
- Implementing variable-frequency drives.
- Reducing the number of air compressors.
- Installing sub-metering and monitoring systems.
- Upgrading lighting and controls.
- Enhancing air recirculation systems.

These measures led to:
- A reduction in overall energy consumption by more than 30%.
- Annual energy cost savings exceeding $11.5 million.

Nissan's commitment to energy efficiency is further demonstrated by its pledge to reduce energy intensity by 2.5% annually as a Save Energy Now LEADER. The company also hosted the first Save Energy Now LEADER Industrial Sustainability and Energy Management Showcase at its Smyrna facility, sharing best practices with over 100 attendees, including industry peers and government officials.
"""
nissan_metadata = {
  "use_case": "reducing energy consumption and costs in automotive manufacturing",
  "solution_type": "implementation of comprehensive energy management program including equipment upgrades and process improvements",
  "impact": "over 30% reduction in energy consumption; annual energy cost savings exceeding $11.5 million",
  "estimated_savings": "more than $11.5 million annually",
  "region": "Smyrna and Decherd, Tennessee; Canton, Mississippi, USA",
  "business_type": "automotive manufacturing"
}

owens_corning_casestudy = """
The article discusses the energy efficiency initiatives implemented at the Owens Corning plant in Santa Clara, California, with the help of the U.S. Department of Energy’s (DOE) Industrial Technologies Program (ITP) and financial incentives from Silicon Valley Power (SVP).

The initiatives included assessments of pumping, fan, and compressed air systems using DOE software tools, leading to upgrades such as installing variable frequency drives (VFDs) on fans and upgrading a large compressor. These efforts resulted in annual energy cost savings of $252,000 for the Santa Clara plant. SVP's rebates, totaling approximately $251,000, played a crucial role in making these projects financially viable, with an overall simple payback period of 1.3 years. The partnership between Owens Corning, DOE, and SVP demonstrates how strategic relationships and financial incentives can drive significant energy savings in industrial facilities.
"""
owens_corning_metadata = {
  "use_case": "improving energy efficiency in an industrial plant",
  "solution_type": "assessments of systems and equipment upgrades (VFDs, compressor)",
  "impact": "annual energy cost savings of $252,000",
  "estimated_savings": "$252,000 annually",
  "region": "Santa Clara, California, USA",
  "business_type": "industrial manufacturing"
}

shaw_industries_casestudy = """
The case study details how Shaw Industries' Plant #20 in Dalton, Georgia, enhanced its steam system efficiency through a Save Energy Now assessment. The U.S. Department of Energy (DOE) conducted the assessment, revealing opportunities for energy savings and validating the plant's existing energy management practices.

Key improvements included:
- Optimizing boiler sequencing
- Installing waste heat exchangers
- Adding an economizer to a boiler

These measures led to:
- Annual energy cost savings of $872,000
- Savings equivalent to 93,000 MMBtu in natural gas and No. 6 fuel oil.
- A simple payback period of 1.7 years on project costs of approximately $1.5 million.

The success of this initiative prompted Shaw Industries to replicate the process at its other facilities.
"""
shaw_industries_metadata = {
  "use_case": "improving steam system efficiency in an industrial plant",
  "solution_type": "steam system assessment and implementation of energy-saving measures",
  "impact": "annual energy cost savings of $872,000, savings equivalent to 93,000 MMBtu",
  "estimated_savings": "$872,000 annually",
  "region": "Dalton, Georgia, USA",
  "business_type": "industrial manufacturing"
}

solutia_casestudy = """
        The case study focuses on how Solutia, Inc. implemented sub-metering across its 24 worldwide manufacturing plants to achieve its sustainability goals, particularly reducing energy intensity.

        Key aspects of Solutia's approach include:
        - Installing sub-meters to measure electricity, steam, and natural gas flow.
        - Utilizing data management software for collecting and analyzing energy consumption data.
        - Using sub-metering data to inform decision-making, maintenance, and equipment selection.

        A specific example at Solutia’s Trenton, Michigan, plant showed that:
        - A $1,500 sub-meter identified an inefficient air compressor.
        - Operational changes based on the sub-meter data led to annual electricity cost savings of $19,000.

        Solutia's experience demonstrates the effectiveness of data-driven approaches in improving energy efficiency and reducing costs.
"""
solutia_metadata = {
  "use_case": "reducing energy intensity through sub-metering",
  "solution_type": "installation and utilization of sub-meters and data management software",
  "impact": "improved energy efficiency and cost savings",
  "estimated_savings": "$19,000 annually at the Trenton plant example",
  "region": "worldwide (example: Trenton, Michigan, USA)",
  "business_type": "manufacturing"
}

volvo_casestudy = """The Volvo Trucks New River Valley (NRV) plant aimed to reduce overall energy consumption by 25% in one year. They partnered with the Industrial Technologies Program (ITP) and established an energy committee to encourage employee engagement.

Specific energy-saving projects and their impacts included:

* **Temperature Reduction:** Turning down the building temperature by 5 degrees resulted in monthly savings of 500,000 kWh and approximately $30,000 in electricity costs.
* **Loading Dock Lights:** Turning off outside loading dock lights at night saved the plant 20,160 kWh each month and $1,200 in monthly electricity costs.
* **Radiant Heating System:** Installing a high-efficiency radiant heating system in the shipping building reduced natural gas usage by 80% and saved 70,000 kWh annually, with a payback period of 2.5 years.

These projects contributed to annual energy savings of over $400,000 and a reduction in annual electricity use by more than 7 gigawatt hours.
"""
volvo_metadata = {
  "use_case": "achieving energy and carbon reduction goals in truck manufacturing",
  "solution_type": "energy assessments, employee engagement, and implementation of energy-saving projects",
  "impact": "25% reduction in energy consumption in one year, over $400,000 annual energy savings, reduction in annual electricity use by more than 7 gigawatt hours",
  "estimated_savings": "over $400,000 annually",
  "region": "Dublin, Virginia, USA",
  "business_type": "truck manufacturing"
}


energy_cases.append(alcoa_casestudy)
energy_cases.append(chrome_deposit_corporation_casestudy)
energy_cases.append(ccp_composites_casestudy)
energy_cases.append(eck_industries_casestudy)
energy_cases.append(flambeau_river_casestudy)
energy_cases.append(freescale_semiconductor_casestudy)
energy_cases.append(harbec_plastics_casestudy)
energy_cases.append(ingersoll_rand_casestudy)
energy_cases.append(minsouth_metalllurgical_casestudy)
energy_cases.append(nissan_casestudy)
energy_cases.append(owens_corning_casestudy)
energy_cases.append(shaw_industries_casestudy)
energy_cases.append(solutia_casestudy)
energy_cases.append(volvo_casestudy)

energy_metadata.append(alcoa_metadata)
energy_metadata.append(chrome_deposit_corporation_metadata)
energy_metadata.append(ccp_composites_metadata)
energy_metadata.append(eck_industries_metadata)
energy_metadata.append(flambeau_river_metadata)
energy_metadata.append(freescale_semiconductor_metadata)
energy_metadata.append(harbec_plastics_metadata)
energy_metadata.append(ingersoll_rand_metadata)
energy_metadata.append(minsouth_metalllurgical_metadata)
energy_metadata.append(nissan_metadata)
energy_metadata.append(owens_corning_metadata)
energy_metadata.append(shaw_industries_metadata)
energy_metadata.append(solutia_metadata)
energy_metadata.append(volvo_metadata)

Steam Case

In [ ]:
boise_inc_casestudy = """
This article discusses how Boise Inc.'s St. Helens paper mill achieved significant fuel savings through a U.S. Department of Energy (DOE) Save Energy Now assessment. The assessment, conducted by DOE Energy Expert David Morgan, aimed to identify natural gas savings opportunities in the mill's steam system. The assessment team used DOE's steam system software tools to analyze plant data and identify potential energy-saving measures.

The assessment validated some measures that were already under consideration, such as lowering the oxygen content on boiler stacks and recovering waste heat from the whitewater process. It also uncovered additional savings opportunities, including repairing a malfunctioning heat exchanger, improving boiler efficiency, and insulating a blow tank. The mill implemented several near-term measures, including lowering package boiler stack O2 levels and recovering waste heat, and identified a new opportunity to reduce steam supplied to a parasitic load.

As a result, the mill achieved annual energy cost savings of more than $1 million and fuel savings of approximately 154,000 MMBtu. The implementation costs were around $31,000, resulting in a simple payback of less than one month. The findings and results were shared with other Boise Inc. paper mills in the United States. The article emphasizes the value of independent evaluations and the use of DOE software tools in identifying energy-saving opportunities in industrial steam systems.
"""
boise_inc_metadata = {
  "use_case": "achieving fuel savings in a paper mill's steam system",
  "solution_type": "steam system assessment and implementation of energy-saving measures",
  "impact": "annual energy cost savings of more than $1 million and fuel savings of approximately 154,000 MMBtu",
  "estimated_savings": "more than $1 million annually",
  "region": "St. Helens, USA",
  "business_type": "paper mill"
}

chrysler_save_energy_now = """
A comprehensive energy assessment was conducted at Chrysler's St. Louis Assembly Complex under the auspices of the U.S. Department of Energy's Save Energy Now initiative. The primary objective of this assessment was to identify and implement measures to reduce natural gas consumption within the facility's steam system.

The assessment process involved a detailed analysis of boiler operations, flue gas composition, and steam trap functionality. Based on the findings, several key recommendations were implemented, including the optimization of boiler sequencing and combustion controls, the reduction of excess oxygen in flue gases, and the establishment of a proactive steam trap maintenance program.

The implementation of these measures resulted in significant energy savings. The St. Louis Assembly Complex achieved an annual reduction in energy consumption exceeding 70,000 MMBtu, yielding annual cost savings of approximately $627,000. The total investment in these energy efficiency upgrades was recouped within a period of slightly over two months. The success of this project has led to the adoption of similar energy management strategies across other Chrysler manufacturing facilities.
"""
chrysler_save_energy_now_metadata = {
  "use_case": "achieving natural gas savings through a plant energy assessment",
  "solution_type": "steam system analysis and implementation of energy-saving measures",
  "impact": "annual energy savings exceeding 70,000 MMBtu and annual cost savings of approximately $627,000",
  "estimated_savings": "approximately $627,000 annually",
  "region": "St. Louis, USA",
  "business_type": "automotive assembly"
}

dow_chemical_casestudy = """
This case study details how Dow Chemical Company's St. Charles Operations petrochemical plant in Hahnville, Louisiana, achieved substantial energy savings through a Save Energy Now energy assessment. The assessment, conducted by a U.S. Department of Energy (DOE) Energy Expert, targeted natural gas savings within the plant's steam system.

The assessment identified several opportunities to enhance steam system efficiency, notably repairing steam leaks and malfunctioning steam traps. Implementing these short-term measures resulted in annual natural gas savings of 272,000 MMBtu and cost savings of $1.9 million. This project boasted a remarkably short payback period of approximately six weeks.

Furthermore, the assessment highlighted medium-term opportunities, such as installing a blowdown heat recovery exchanger and preheating reactor feed with 75-psig steam, which promise further energy and cost reductions. The DOE's Steam System Assessment Tool (SSAT) played a crucial role in quantifying potential savings and motivating the plant to implement these improvements. The success of these initiatives has led to the establishment of ongoing steam trap maintenance and leak management programs at the St. Charles plant.
"""
dow_chemical_metadata = {
  "use_case": "achieving natural gas savings in a petrochemical plant's steam system",
  "solution_type": "steam system assessment and implementation of energy-saving measures",
  "impact": "annual natural gas savings of 272,000 MMBtu and cost savings of $1.9 million",
  "estimated_savings": "$1.9 million annually",
  "region": "Hahnville, Louisiana, USA",
  "business_type": "petrochemical plant"
}

goodyear_casestudy = """
This case study details how Goodyear's tire plant in Union City, Tennessee, achieved significant energy savings through a Save Energy Now assessment conducted by the U.S. Department of Energy (DOE). The assessment focused on the plant's steam system, identifying opportunities to improve efficiency and reduce energy consumption.

The key initiatives included optimizing boiler operation and load management, and insulating process equipment. By implementing these recommendations, Goodyear reduced natural gas use by 93,000 MMBtu and No. 6 fuel oil consumption by 224,000 gallons, resulting in annual energy cost savings of approximately $875,000. The project had a simple payback period of just 2.5 months, demonstrating the cost-effectiveness of the energy-saving measures. The success of this assessment has prompted Goodyear to share these energy-saving recommendations with its other facilities.
"""
goodyear_metadata = {
  "use_case": "achieving energy savings in a tire plant's steam system",
  "solution_type": "steam system assessment and implementation of energy-saving measures",
  "impact": "reduced natural gas use by 93,000 MMBtu and No. 6 fuel oil consumption by 224,000 gallons, resulting in annual energy cost savings of approximately $875,000",
  "estimated_savings": "approximately $875,000 annually",
  "region": "Union City, Tennessee, USA",
  "business_type": "tire manufacturing"
}

jr_simplot_casestudy = """
This case study discusses how J.R. Simplot Company improved the efficiency of its steam system at a potato processing plant in Caldwell, Idaho, leading to significant energy and cost savings.

The project, based on an evaluation using the Steam System Assessment Tool (SSAT), involved upgrading burners and burner control systems. This resulted in annual savings of 52,000 MMBtu of natural gas and 526,000 kWh of electricity, totaling $329,000. The project achieved a simple payback of less than 14 months.

Following the success at the Caldwell facility, Simplot implemented similar measures at eight other facilities, yielding additional natural gas savings. The company also mandated the use of SSAT for evaluating steam systems across all its production facilities. The case study highlights the effectiveness of using a uniform methodology and tool for optimizing industrial steam systems.
"""
jr_simplot_metadata = {
    "use_case": "Improving steam system efficiency in a food processing plant",
    "solution_type": "Upgrading burners and burner control systems",
    "impact": "Reduced energy consumption and cost",
    "estimated_savings": "$329,000 per year at the Caldwell plant, plus additional savings from replicating the project at other facilities",
    "region": "Caldwell, Idaho, United States",
    "business_type": "Food processing (potato processing plant)"
}

westlinn_paper_casestudy =  """
This case study details how West Linn Paper Company, the oldest active paper mill in the western United States (located in Oregon), achieved significant natural gas savings through a Save Energy Now assessment conducted by the U.S. Department of Energy (DOE). The assessment focused on improving the efficiency of the mill's steam system, which is critical for drying paper and powering papermaking machines.

By implementing key recommendations, including connecting separate steam headers, adjusting boiler controls, and repairing steam traps, the mill achieved annual natural gas savings of approximately 58,200 MMBtu, resulting in annual cost savings of $379,000. Furthermore, the assessment empowered plant employees with the knowledge to analyze their steam processes and identify future energy-saving opportunities. The mill's daily production is nearly 700 tons of coated paper. Since improving steam system efficiency, the mill's energy use declined from 7,500 to 5,200 lbs/ton of paper and production increased by 40%. The project had a payback period of under 6 months. At the time of the assessment, the steam system generated approximately 166,000 lbs/hour at 200 psig.
"""
westlinn_paper_metadata = {
  "use_case": "improving steam system efficiency in a paper mill",
  "solution_type": "steam system assessment and implementation of energy-saving measures",
  "impact": "annual natural gas savings of approximately 58,200 MMBtu, annual cost savings of $379,000",
  "estimated_savings": "approximately $379,000 annually",
  "region": "West Linn, Oregon, USA",
  "business_type": "paper mill"
}

jr_simplot_steam_casestudy = """
This case study focuses on how J.R. Simplot Company's Don Plant in Pocatello, Idaho, optimized its steam system efficiency following a U.S. Department of Energy (DOE) Save Energy Now assessment in 2006. The assessment, led by DOE Energy Expert Bill Moir, utilized DOE's Steam System Assessment Tool (SSAT) software to pinpoint multiple energy-saving opportunities.

By implementing recommendations such as optimizing boiler operation, improving condensate recovery, and repairing steam traps and leaks, the plant achieved significant energy savings. These measures led to annual cost savings of $335,000 and energy savings exceeding 75,000 MMBtu. The project, costing $180,000, had a simple payback period of approximately 6.5 months.

The success of these initial improvements spurred the plant to investigate further energy-saving measures and share their insights with other J.R. Simplot facilities.
"""
jr_simplot_steam_metadata = {
  "use_case": "optimizing steam system efficiency in a fertilizer plant",
  "solution_type": "steam system assessment and implementation of energy-saving measures",
  "impact": "annual cost savings of $335,000 and energy savings of more than 75,000 MMBtu",
  "estimated_savings": "$335,000 annually",
  "region": "Pocatello, Idaho, USA",
  "business_type": "fertilizer plant"
}

terra_nitrogen_casestudy = """
This case study details how Terra Nitrogen Company, L.P., achieved significant natural gas savings at its ammonia and fertilizer plant in Verdigris, Oklahoma, following a U.S. Department of Energy (DOE) Save Energy Now assessment. The assessment, conducted by a DOE Energy Expert, identified opportunities to enhance the plant's steam system efficiency.

Key improvements included:
- Upgrading steam-powered turbines
- Modifying an ammonia condensing loop
- Repairing steam traps and leaks

These measures resulted in annual natural gas savings of 497,000 MMBtu and approximately $3.5 million in cost savings. The company shared the assessment results with three other U.S. plants owned by its parent company.
"""
terra_nitrogen_metadata = {
  "use_case": "reduction of natural gas consumption in an ammonia plant's steam system",
  "solution_type": "upgrading steam turbines, modifying an ammonia condensing loop, and repairing steam traps and leaks",
  "impact": "annual natural gas savings of 497,000 MMBtu",
  "estimated_savings": "approximately $3.5 million annually",
  "region": "Verdigris, Oklahoma, USA",
  "business_type": "manufacturer of nitrogen fertilizer products"
}

energy_cases.append(boise_inc_casestudy)
energy_cases.append(chrysler_save_energy_now)
energy_cases.append(dow_chemical_casestudy)
energy_cases.append(goodyear_casestudy)
energy_cases.append(jr_simplot_casestudy)
energy_cases.append(westlinn_paper_casestudy)
energy_cases.append(jr_simplot_steam_casestudy)
energy_cases.append(terra_nitrogen_casestudy)

energy_metadata.append(boise_inc_metadata)
energy_metadata.append(chrysler_save_energy_now_metadata)
energy_metadata.append(dow_chemical_metadata)
energy_metadata.append(goodyear_metadata)
energy_metadata.append(jr_simplot_metadata)
energy_metadata.append(westlinn_paper_metadata)
energy_metadata.append(jr_simplot_steam_metadata)
energy_metadata.append(terra_nitrogen_metadata)

In [ ]:
energy_case_metadata = []
energy_case_id = []
for energy_case in energy_cases:
  energy_case_metadata.append({
        "purpose": "case_study",
        "domain": "inventory",
  })
  energy_case_id.append(str(uuid.uuid4()))

energy_collection.add(
  documents=energy_cases,
  metadatas=energy_case_metadata,
  ids=energy_case_id)

### --- Energy Case Studies ---

In [ ]:
kaiser_aluminum = '''
This case study discusses how a U.S. Department of Energy-sponsored (DOE) Save Energy Now assessment helped Kaiser Aluminum improve energy efficiency and productivity at its aluminum extrusion plant in Sherman, Texas. The assessment, conducted in 2006, utilized DOE's Process Heating Assessment and Survey Tool (PHAST) software to analyze the plant's process heating systems.

The assessment focused on identifying opportunities to improve furnace efficiency and reduce heat loss in the process heating applications of secondary aluminum plants. The solutions implemented included adjusting burner controls on a reverberatory melting furnace to lower excess oxygen levels and repairing the furnace's door sill and jamb to prevent cold air infiltration.

Implementing these measures resulted in significant natural gas savings, improved furnace energy intensity, and enhanced production. The plant achieved annual energy savings of approximately 45,000 MMBtu and improved the furnace's energy intensity by 11.1% between 2006 and 2007. The energy cost savings amounted to $360,000, with a simple payback period of less than 1 month.
'''
lean_dna = '''
This case study discusses how LeanDNA helped a leading aerospace manufacturer reduce inventory costs and improve supply chain efficiency. The manufacturer, struggling with labor-intensive Excel spreadsheets to manage their supply chain activities after a major SAP implementation, sought a solution to streamline operations and reduce inventory.

LeanDNA was implemented and provided pre-built dashboards that visualized data from SAP, offering insights into inventory improvement opportunities. This allowed the manufacturer to focus on high-impact initiatives, reducing inventory and working capital. The solution enhanced visibility across different sites and ERP systems, consolidating data into a single view.
'''
hvac_system = """
In modern building operations, HVAC systems play a critical role in ensuring occupant comfort and energy efficiency. A recent study published in *Energy and Buildings* examines the tangible benefits of HVAC system cleaning, focusing on how routine maintenance impacts energy consumption and supply airflow.

The research explores real-world scenarios where HVAC systems were subject to systematic cleaning procedures. Results indicated measurable improvements in energy efficiency and airflow consistency, supporting the hypothesis that neglected systems often operate suboptimally. Clean components such as coils and ducts contribute to reduced fan power demand and better thermal performance.

This study reinforces the value of preventive maintenance, not only as a means of prolonging equipment lifespan but also as a viable strategy for enhancing sustainability in commercial and residential buildings. Facilities managers and energy engineers are encouraged to integrate cleaning into standard operational protocols to achieve long-term energy savings and improved indoor air quality.
"""

energy_collection.add(
    documents = [kaiser_aluminum, lean_dna, hvac_system],
    metadatas = [
        {"purpose": "case_study", "domain": "inventory"},
        {"purpose": "case_study", "domain": "inventory"},
        {"purpose": "case_study", "domain": "energy"}
    ],
    ids = [
        str(uuid.uuid4()),
        str(uuid.uuid4()),
        str(uuid.uuid4())
    ]
)




### Run a similarity search: example code

In [ ]:
# Swtich to query mode
embed_fn.document_mode = False

# Define query
query = "How can I reduce food waste in my restaurant?"

# Search
results = inventory_collection.query(
    query_texts=[query],
    n_results=2
)

# Display results
for i, doc in enumerate(results["documents"][0]):
    print(f"🔍 Match {i+1}: {doc}")
    print(f"📄 Metadata: {results['metadatas'][0][i]}")
    print("-" * 60)

## -- LangGraph Workflow Overview --

In [ ]:
# [🔁 LangGraph Agent Workflow]
#    ├── 🧭 Node 1: Classifier
#    │     → Classifies user input as:
#    │        • Inventory Waste
#    │        • Energy Inefficiency
#    │        • Default / No Issue
#    │
#    ├── 📂 Node 2: RAG Retriever – Energy Inefficiency
#    │     → Retrieves similar energy cases, cost metrics, solution examples from ChromaDB
#    │
#    ├── 📦 Node 3: RAG Retriever – Inventory Waste
#    │     → Retrieves similar inventory cases, benchmarks, inefficiency insights from ChromaDB
#    │
#    ├── 💸 Node 4: Inaction Cost Estimator
#    │     → Estimates the financial cost of not addressing the identified inefficiency
#    │        (based on retrieved case data + heuristics)
#    │
#    ├── 🛠️ Node 5: Solution Generator
#    │     → Recommends cost-effective, high-ROI interventions aligned with SDGs
#    │     → (Optional) Uses RAG to refine suggestions
#    │
#    ├── 🧾 Node 6: Final Report Composer
#    │     → Generates structured business-facing report:
#    │        Problem → Cost of Inaction → Solutions → ROI
#    │
#    ├── 📊 Node 7: Metrics & Performance Generator [Default Case]
#    │     → If no inefficiency is found:
#    │        Calculates current operational performance and business metrics
#    │
#    └── 📋 Node 8: Alternative Report Composer [Default Case]
#          → Generates a report with:
#             • Current benefits
#             • Key performance indicators
#             • Strategic outlook and future potential


### --- Define Structure ---

In [ ]:
class AgentState(TypedDict):
    input: str
    issue_type: Optional[Literal["inventory", "energy", "default"]]
    retrieved_docs: Optional[Dict[str, List[Document]]]
    inaction_summary: Optional[str]
    solutions: Optional[str]
    report: Optional[str]

In [ ]:
# Initialize the graph where we'll register our nodes
workflow = StateGraph(AgentState)

In [ ]:
# Function for effectively querying the DB
def retrieve_similar_cases(query: str, database, client, purposes: List[str] = ["case_study", "cost_estimates", "solution", "mixed"], domain: str = None, top_k: int = 5):
    embed_query = GeminiEmbeddingFunction(client=client, document_mode=False)
    query_embedding = embed_query([query])[0]

    results = {}

    for purpose in purposes:
        filter_dict = {
            "$and": [
                {"purpose": "cost_estimates"},
                {"domain": "energy"}
            ]
        }
        results = database.query(
            query_embeddings=[query_embedding],
            n_results=top_k,
            where=filter_dict,
            include=["documents", "metadatas"]
        )

        # Store as list of LangChain Documents
        docs = [
            Document(page_content=doc, metadata=meta)
            for doc, meta in zip(results["documents"][0], results["metadatas"][0])
        ]
        results[purpose] = docs

    return results


### --- Define Agent Nodes ---
#### ---- Classifier ----
Identifies the business inefficiency category from user input (inventory waste, energy inefficiency, or none). Routes the request accordingly.

In [ ]:
input_prompt = PromptTemplate.from_template("""
You are a business analyst that helps identify problems in energy efficiency and inventory mismanagement that could be costing the business.

Based on the given business profile and question from the owner, please identify the problem
Identify the issue as either:
- "energy": when the main problem faced by the business is related to energy consumption, heating/cooling systems, lighting, electricity, etc.
- "inventory": when the main business problem comes from issues related to stock, storage, waste, overstock, ordering, losses, inventory tracking, etc.
- "default": if the business seems to be in a good standing

Only output one word: "energy", "inventory", or "default"

Business profile and user input:
{input}
""")

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [ ]:
# Node 1: Classifier
def classifier(state: AgentState) -> AgentState:
    # Format the prompt using the user's input
    formatted_prompt = input_prompt.format(input=state["input"])
    result = llm.invoke(formatted_prompt)
    issue_type = result.content.strip().lower()

    # Sanity check (fallback)
    if issue_type not in ["energy", "inventory", "default"]:
        issue_type = "default"

    return {**state, "issue_type": issue_type}


#### ---- RAG for Energy ----
Retrieves relevant documents from ChromaDB:
> Energy cases → efficiency benchmarks, inaction examples

In [ ]:
# Node 2: RAG Retriever for Energy
def retrieve_energy_docs(state: AgentState) -> AgentState:
    if state["issue_type"] != "energy":
        return state

    query = state["input"]
    results = retrieve_similar_cases(
        query=query,
        database=energy_collection,
        client=client,
        domain="energy"
    )

    return {**state, "retrieved_docs": results}

#### ---- RAG for Inventory ----
Retrieves relevant documents from ChromaDB:
> Inventory cases → waste metrics, intervention results

In [ ]:
# Node 3: RAG Retriever for Inventory
def retrieve_inventory_docs(state: AgentState) -> AgentState:
    if state["issue_type"] != "inventory":
        return state

    query = state["input"]
    results = retrieve_similar_cases(
        query=query,
        database=inventory_collection,
        client=client,
        domain="inventory"
    )

    return {**state, "retrieved_docs": results}

#### ---- Cost of Inaction Estimator ----
Predicts monetary loss associated with not addressing the inefficiency, using analogies from similar past cases and embedded metadata.

In [ ]:
# Define a structured output schema
class InactionCost(BaseModel):
    estimated_loss: str = Field(description="Estimated Loss")
    contributing_factors: List[str] = Field(description="Contributing Factors")
    summary: str = Field(description="Summary")

In [ ]:
inaction_prompt_template = PromptTemplate.from_template("""
You are a cost analyst for sustainability inefficiencies. Your goal is to help businesses quantify the financial risk of inefficiencies.

Given the following user input and retrieved case examples,
- Estimate the potential financial loss or operational impact (dollar value) of **not taking action** and letting the inefficiency continue
- Identify key contributing factors
- Write a 1-paragraph natural language summary about the operational impact

User Input:
{user_input}

Retrieved Case Summaries:
{docs}
""")

In [ ]:
structured_llm = llm.with_structured_output(InactionCost)

In [ ]:
def estimate_inaction_cost(state: AgentState) -> AgentState:
    user_input = state["input"]
    retrieved = state.get("retrieved_docs", {})
    cost_estimate_docs = retrieved.get("cost_estimates", [])

    if cost_estimate_docs:
        doc_texts = "\n\n".join([doc.page_content for doc in cost_estimate_docs])
    else:
        doc_texts = "No cost-estimate cases were found in the database."

    prompt = inaction_prompt_template.format(
        user_input=user_input,
        docs=doc_texts
    )

    inaction_summary = structured_llm.invoke(prompt)

    return {**state, "inaction_summary": inaction_summary}

#### ---- Solution Generator ----
Recommends tailored, SDG-aligned, and actionable interventions. Focuses on those with proven ROI and potential business uplift.

In [ ]:
# Define structured output schema
class SolutionRecommendation(BaseModel):
    strategies: List[str] = Field(description="Strategies")
    justification: str = Field(description="Justification")
    benefits: str = Field(description="Benefits")
    sdg_alignment: str = Field(description="Alignment with SDGs")

In [ ]:
solution_prompt_template = PromptTemplate.from_template("""
You are an expert sustainability advisor for operational inefficiencies and best business practices.

Based on the business profile, the operational inefficiency identified, the relevant information retrieved, recommend **high-impact, cost-effective strategies** for solving the problem.
Solutions should be tailored to the business.

Respond with:
- 3 to 5 specific strategies and a description
- Why it's effectice (evidence, explanation, or retrieved example)
- List the potential financial gains or operational impact, in estimated dollar $ value, as well as estimated benefits (financial, operational, economic, and environmental) and/or ROI
- Relevant SDG (Sustainable Development Goals) alignment (if applicable) (an explanation)

## Business Profile:
{business_profile}

## Inefficiency:
{issue_type}

Retrieved knowledge:
{docs}
""")


In [ ]:
structured_solution_llm = llm.with_structured_output(SolutionRecommendation)

In [ ]:
# Node 5: Solution Generator
def generate_solutions(state: AgentState) -> AgentState:
    business_profile = state["input"]
    issue_type = state["issue_type"]
    user_input = state["input"]
    docs = state.get("retrieved_docs", [])
    cost_estimate_docs = docs.get("solution", [])

    doc_texts = "\n\n".join([doc.page_content for doc in cost_estimate_docs])

    prompt = solution_prompt_template.format(
        user_input=user_input,
        business_profile=business_profile,
        issue_type=issue_type,
        docs=doc_texts
    )

    result = structured_solution_llm.invoke(prompt)

    return {**state, "solutions": result}

#### ---- Final Report Composer ----
Crafts a concise, structured report highlighting:
- Identified problem
- Estimated cost of inaction
- Recommended interventions
- Expected ROI and benefits

In [ ]:
def compose_final_report(state: AgentState) -> AgentState:
    # Extract and format input summary
    user_input = state["input"]
    if isinstance(user_input, dict):  # structured input
        input_summary = "\n".join([f"- **{format_key(k)}:** {format_value(v)}" for k, v in user_input.items()])
    else:
        input_summary = user_input.strip()

    # Clean report sections
    estimated_loss = state['inaction_summary'].estimated_loss.strip()
    contributing_factors = "\n".join([f"   - {factor.strip()}" for factor in state['inaction_summary'].contributing_factors])
    impact_summary = state['inaction_summary'].summary.strip()

    strategies = "\n".join([f"- {s.strip()}" for s in state['solutions'].strategies])
    benefits = state['solutions'].benefits.strip()
    justification = state['solutions'].justification.strip()
    sdgs = state['solutions'].sdg_alignment.strip()

    # Compose final  report
    report = f"""Efficiency Report

Problem Type: {state['issue_type'].capitalize()}

📋 Business Input Summary:

{user_input}

---

🔍 Estimated Cost of Inaction

- Estimated Loss: {estimated_loss}
- Contributing Factors:
{contributing_factors}

Impact Summary:
{impact_summary}

---

✅ Recommended Solutions

Top Strategies:
{strategies}

Expected Benefits:
{benefits}

Justification:
{justification}

Alignment with SDGs:
{sdgs}
"""

    return {**state, "report": report}


### --- Performance & Metrics Generator ---
Calculates current strengths and metrics of the business (e.g., turnover rate, waste %, energy use benchmarks).

In [ ]:
# Define structured output schema
class PerformanceGeneration(BaseModel):
    benefits: List[str] = Field(description="Benefits")
    metrics: str = Field(description="Metrics & Business Performance")
    outlook: str = Field(description="Outlook")

In [ ]:
performance_generation_prompt_template = PromptTemplate.from_template("""
You are an expert sustainability advisor for operational inefficiencies and best business practices.

Based on the business profile, the operational inefficiency identified, the relevant information retrieved, recommend **high-impact, cost-effective strategies** for solving the problem.
Solutions should be tailored to the business.

Respond with:
- Estimated benefits stemming from current inventory optimization & energy consumption (financial, operational, economic, and environmental) (in dollar value)
- Current business metrics & performance (from business profile, and knowledge of similar businesses)
- Future Outlook (an explanation on why so)

## Business Profile:
{structured_input}

""")

In [ ]:
performance_metrics_llm = llm.with_structured_output(PerformanceGeneration)

In [ ]:
# Node 7: Performance & Metrics Generator
def generate_performance_metrics(state: AgentState) -> AgentState:
    user_input = state["input"]

    prompt = performance_generation_prompt_template.format(
        user_input=user_input,
    )

    result = performance_metrics_llm.invoke(prompt)

    return {**state, "performance_metrics": result}

### --- Alternate Report Composer ---
Produces a future-oriented report even when no major inefficiency is detected, emphasizing current operational health and future scaling insights.

In [ ]:
def compose_alternate_report(state: AgentState) -> AgentState:
    report = f"""
    # Efficiency Report

    **Problem Type:**  All Good!!!
    **Input Summary:** {state['input']}

    ---

    ## ✅ Estimated Benefits from current inventory optimization & energy consumption
    {chr(10).join(f"  - {benefit}" for benefit in state['performance_metrics'].benefits)}

    ---

    ## 📈 Metrics & Business Performance
    {state['performance_metrics'].metrics}

    ---

    ## 🔮 Future Outlook
    {state['performance_metrics'].outlook}
    """

    return {**state, "report": report}

### --- Branching Logic for Retrieval Nodes ---

In [ ]:
# Add nodes to the graph object
workflow.add_node("classifier", classifier)
workflow.add_node("energy_retriever", retrieve_energy_docs)
workflow.add_node("inventory_retriever", retrieve_inventory_docs)
workflow.add_node("cost_estimator", estimate_inaction_cost)
workflow.add_node("solution_generator", generate_solutions)
workflow.add_node("report_composer", compose_final_report)

workflow.add_node("performance_generator", generate_performance_metrics)
workflow.add_node("alternate_report_composer", compose_alternate_report)

In [ ]:
def route_retriever(state: AgentState) -> str:
    if state["issue_type"] == "energy":
        return "energy_retriever"
    elif state["issue_type"] == "inventory":
        return "inventory_retriever"
    else:
        return "default"

In [ ]:
# Add conditional branch routing after classifier
workflow.add_conditional_edges(
    "classifier",
    route_retriever,
    {
        "energy_retriever": "energy_retriever",
        "inventory_retriever": "inventory_retriever",
        "default": "performance_generator"
    }
)

In [ ]:
# Define fixed edges between remaining nodes
workflow.add_edge(START, "classifier")
workflow.add_edge("energy_retriever", "cost_estimator")
workflow.add_edge("inventory_retriever", "cost_estimator")
workflow.add_edge("cost_estimator", "solution_generator")
workflow.add_edge("solution_generator", "report_composer")
workflow.add_edge("report_composer", END)

workflow.add_edge("performance_generator", "alternate_report_composer")
workflow.add_edge("alternate_report_composer", END)

## -- Running the Full Pipeline --

In [ ]:
app = workflow.compile()

### --- Input ---
The user provides:
- A structured business profile (industry, size, energy usage, etc.)
- A natural language description of operations or goals


In [ ]:
# Example input 1
business_name = "EcoPrint Manufacturing Co."
industry = "Light Manufacturing – Sustainable Packaging"
location = "Mississauga, Canada"
annual_revenue = "$5,800,000"
number_of_employees = 42
physical_space_type = "mid-size industrial facility"
has_online_sales = False
inventory_management_method = "ERP system"
inventory_turnover_frequency = "monthly"
energy_systems = {
    "heating": "natural gas unit heaters",
    "cooling": "none",
    "lighting": "fluorescent tubes across the floor",
    "automation": "basic motion sensors in offices only"
}
monthly_energy_cost = "$7,600"
existing_sustainability_practices = ["waste separation", "recycled paper sourcing"]

user_prompt = (
    "We manufacture custom biodegradable packaging. Our machines run long hours and our energy bills are massive—especially during the winter. "
    "Most of our lighting is outdated and stays on all day across the warehouse floor. "
    "We’d like to explore ways to optimize energy usage and get better visibility into where our waste is happening."
)


In [ ]:
business_schema = f"""
- Business Name: {business_name}
- Industry: {industry}
- Location: {location}
- Annual Revenue: {annual_revenue}
- Number of Employees: {number_of_employees}
- Physical Space Type: {physical_space_type}
- Online Presence: {has_online_sales}
- Inventory Management Method: {inventory_management_method}
- Inventory Turnover Frequency: {inventory_turnover_frequency}
- Energy Systems: {energy_systems}
- Monthly Energy Costs: {monthly_energy_cost}
- Existing Sustainability Practices: {existing_sustainability_practices}
- Question from Business Owner: {user_prompt}
"""

In [ ]:
# Example 1
result = app.invoke({"input": business_schema})
print(result["report"])



In [ ]:
# Example Input 2
business_name = "Bottle & Cork Wine Merchants"
industry = "Retail – Specialty Liquor"
location = "Vancouver, Canada"
annual_revenue = "$2,400,000"
number_of_employees = 10
physical_space_type = "retail storefront with on-site storage"
has_online_sales = True
inventory_management_method = "manual spreadsheet"
inventory_turnover_frequency = "quarterly"
energy_systems = {
    "heating": "electric baseboard",
    "cooling": "central A/C",
    "lighting": "halogen",
    "automation": "none"
}
monthly_energy_cost = "$950"
existing_sustainability_practices = ["glass bottle recycling", "LED signs in entrance"]

user_prompt = (
    "We’ve been noticing a lot of product waste—seasonal wines that don’t sell in time, "
    "and some bottles even spoil on the shelf. We manually update inventory, so sometimes items get missed. "
    "We want to reduce waste and improve our shelf planning. We also think our lighting might be driving up energy costs."
)


In [ ]:
business_schema = f"""
- Business Name: {business_name}
- Industry: {industry}
- Location: {location}
- Annual Revenue: {annual_revenue}
- Number of Employees: {number_of_employees}
- Physical Space Type: {physical_space_type}
- Online Presence: {has_online_sales}
- Inventory Management Method: {inventory_management_method}
- Inventory Turnover Frequency: {inventory_turnover_frequency}
- Energy Systems: {energy_systems}
- Monthly Energy Costs: {monthly_energy_cost}
- Existing Sustainability Practices: {existing_sustainability_practices}
- Question from Business Owner: {user_prompt}
"""

In [ ]:
# Example 2
result = app.invoke({"input": business_schema})
print(result["report"])

## Conclusion

This advisory tool helps businesses assess operational inefficiencies not just as technical problems, but as financially and socially impactful issues. By quantifying the cost of inaction and aligning solutions with SDGs, it bridges sustainability and strategy.
